In [1]:
# UT-TOR-DATA-PT-01-2020-U-C Team project #1
# (c) Boris Smirnov

# The map of Per Capita Income per Riding for 2015 elections

Legend:

* Red - Liberal Party of Canada

* Blue - Conservative Party of Canada

* Orange - New Democratic Party

* Light Blue - Bloc Québécois

* Green  - Green Party of Canada

The relative size of the circle corresponds to the average per capita income for the riding.

# Observations

Here are some observations, that one could make after a glance on the map:

1. "Red" centers of large agglomerations: Toronto, Montreal, Ottawa, Vancouver, Calgary.

The map demostrates, that downtowns of major canadian cities voted for Liberals. Another interesting thing is that the population of those federal electoral districts have very high income (measured both by amount of taxable income and per capita income). It seems, that center-left agenda of Liberal Party is attractive to well-off inhabitants of expensive properties of city-centres.

Toronto | Montreal
------- | --------
![Toronto](PerCapita_Toronto.png)|![Montreal](PerCapita_MontrealCentre.png)

Vancouver | Calgary
--------- | --------
![Vancouver](PerCapita_Vancouver.png)|![Calgary](PerCapita_Calgary.png)


2. NDP had significant support in BC, it is third (percentwise) after NWT and Nunavut. Combined with the Liberals they overcome Conservatives 2-by-1. Relatively low per capita income correlates well with left agenda of NDP.


3. The opposite example that proves the point is the province of Alberta with it's natural resources and the highest GDP per capita in Canada (https://en.wikipedia.org/wiki/Alberta#Economy). 59.6% votes in favour of the Conservatives vs. 36.1% for Liberals and NDP, practically 2-by-1, diametrally opposed picture compared to BC. Though, central ridings of Edmonton and Calgary are Red and Orange.


Alberta | Windsor
------- | --------
![Alberta](PerCapita_Alberta.png)|![Windsor](PerCapita_Windsor.png)


4. Another notable area of NDP support is in Winsor, ON. It might indicate this part of Ontario probably have economical problems tied, for example, with automotive industry.


5. In Quebec the main rival of NDP was Bloc Québécois. They have similar leftish agenda, but different vision of the future, namely secession from Canada. Otherwise, they attract similar category of low income people.

![Montreal](PerCapita_Montreal.png)

In [2]:
# Dependencies and initialization
import gmaps
import pandas as pd

# Google developer API key
from api_keys import g_key

# Data sources
geodata_csv = 'geodata.csv' # FED Coordinates (key: 'FED Id')
provinces_csv = 'provinces.csv' # Province Ids and coordinates (key: 'Province Id')
parties_csv = '../data/winning_parties_2015.csv' # List of winning parties (key: 'Custom Name')
results_csv = '../data/final_elected.csv' # List of election results by FED (keys: 'FED ID', 'Party Won')
income_csv = '../data/income.csv' # FED Income Data (key: 'FED ID')

In [3]:
# Read data
geodata_df = pd.read_csv(geodata_csv).loc[:, ['FED Id', 'FED Name', 'Province Id', 'Latitude', 'Longitude']]
provinces_df = pd.read_csv(provinces_csv)
parties_df = pd.read_csv(parties_csv).loc[:, ['Id', 'Custom Name']]
results_df = pd.read_csv(results_csv).loc[:, ['FED ID', 'Population', 'Party Won']]
income_df = pd.read_csv(income_csv, encoding='windows-1252').loc[:, ['Prov/Terr', 'FED ID', 'Total Income']]
prov_income_df = income_df[income_df['Prov/Terr'] == 'TOTAL'].copy()
income_df = income_df[income_df['Prov/Terr'] != 'TOTAL']

In [4]:
# Functions to shape symbol_layer dots: size and color

# From the pd.Series s with float values calculate the scale for each point
def make_scale_list(s, bin_count=10): # bin_count=10 seems just fine. If it's 15, the circles are too big and the map is a mess
    interval_index = pd.interval_range(start=s.min() * 0.999, end=s.max(), periods=bin_count)
    return [1 + 1 * int(interval_index.get_loc(val)) for val in s]

# From pd.Series s with Party Ids make a list of party colors
def make_color_list(s):
    # Source: https://www.rapidtables.com/web/color/RGB_Color.html
    # Alternative colors are from wikipedia articles about the political parties
    c_blue      = (51, 51, 255)  # (100, 149, 237)
    c_lightblue = (51, 153, 255) # (135, 206, 250)
    c_green     = (0, 204, 0)    # (153, 201, 85)
    c_orange    = (255, 165, 0)  # (244, 164, 96)
    c_red       = (255, 0, 0)    # (234, 109, 106)

    # Source: https://en.wikipedia.org/wiki/Political_colour
    # Keys are from winning_parties_2015.csv For details see commit 38ab6ec6ab572f326f327a7fb1c8b68740c47ceb
    party_colors = {
        'CPC': c_blue,
        'BQ' : c_lightblue,
        'GRN': c_green,
        'NDP': c_orange,
        'LIB': c_red
    }

    return s.map(lambda id: party_colors[id]).tolist()

In [5]:
# Create data set for FEDs
feds_df = pd.merge(geodata_df, results_df, how='left', left_on='FED Id', right_on='FED ID')
feds_df = pd.merge(feds_df, parties_df, how='left', left_on='Party Won', right_on='Custom Name')
feds_df = pd.merge(feds_df, income_df, how='left', left_on='FED Id', right_on='FED ID')
feds_df.rename(columns={'Id': 'Party Id'}, inplace=True)
feds_df['Per Capita'] = feds_df['Total Income'] / feds_df['Population']
feds_df['Average Per Capita Income'] = feds_df['Per Capita'].map("${:,.0f}".format) # Formatted for hover text
feds_df.drop(columns=['FED ID_x', 'FED ID_y', 'Custom Name', 'Prov/Terr'], inplace=True)
feds_df.head()

,FED Id,FED Name,Province Id,Latitude,Longitude,Population,Party Won,Party Id,Total Income,Per Capita,Average Per Capita Income
0,11001,Cardigan,11,46.251413,-62.652435,36005,Liberal,LIB,1.237610e+09,34373.281489,"$34,373"
1,11002,Charlottetown,11,46.267987,-63.143870,34562,Liberal,LIB,1.192487e+09,34502.835484,"$34,503"
2,24056,Pierrefonds--Dollard,24,45.491925,-73.855406,108740,Liberal,LIB,3.919965e+09,36048.970020,"$36,049"
3,24057,Pontiac,24,46.760139,-76.594503,106499,Liberal,LIB,4.177035e+09,39221.354191,"$39,221"
4,24058,Portneuf--Jacques-Cartier,24,47.137004,-71.836506,104394,Conservative,CPC,4.458991e+09,42713.096538,"$42,713"


In [6]:
# Need to fix prov_income_df: no province ids, no names, only numbers :((
# The file was made from Excel pivot table
prov_income_df

,Prov/Terr,FED ID,Total Income
7,TOTAL,NaN,1.917592e+10
12,TOTAL,NaN,4.704650e+09
24,TOTAL,NaN,3.128133e+10
35,TOTAL,NaN,2.441427e+10
114,TOTAL,NaN,2.797480e+11
236,TOTAL,NaN,5.181170e+11
251,TOTAL,NaN,4.194699e+10
266,TOTAL,NaN,4.067702e+10
301,TOTAL,NaN,1.765550e+11
344,TOTAL,NaN,1.800390e+11


In [7]:
# Create dataset for Provinces

# The last line is Total for Canada. Don't need it.
if len(prov_income_df) > len(provinces_df): # Delete line only once
    prov_income_df = prov_income_df.iloc[0:-1, :]

# Fortunately, the data in prov_income_df is sorted by prov_id
# We can borrow the index from provinces_df
prov_income_df.set_index(provinces_df['Province Id'], inplace=True)

prov_data_grp = feds_df.groupby('Province Id')

# Getting province populations
prov_population_s = prov_data_grp['Population'].sum()
prov_population_s.name = 'Total Population'

# Getting median value for FEDs Per Capita Income for each province
prov_med_percapita_s = prov_data_grp['Per Capita'].median()
prov_med_percapita_s.name = 'Median Per Capita among Province FEDs'

# Making resulting dataset good for display
provs_df = pd.merge(provinces_df, prov_income_df, how='left', left_on='Province Id', right_index=True)
provs_df = pd.merge(provs_df, prov_population_s, how='left', left_on='Province Id', right_index=True)
provs_df = pd.merge(provs_df, prov_med_percapita_s, how='left', left_on='Province Id', right_index=True)
provs_df.drop(columns=['Prov/Terr', 'FED ID'], inplace=True)
provs_df['Population'] = provs_df['Total Population'].map("{:,d}".format)  # Formatted for info boxes
provs_df['Per Capita'] = provs_df['Total Income'] / provs_df['Total Population']
provs_df['Average Per Capita Income'] = provs_df['Per Capita'].map("${:,.0f}".format) # Formatted for info boxes
provs_df['Avg Scale'] = [4, 3, 3, 3, 3, 4, 3, 4, 5, 4, 5, 5, 3] # This is lame :( Should make it calculable
provs_df['Median FED PCI'] = provs_df['Median Per Capita among Province FEDs'].map("${:,.0f}".format) # Formatted for info boxes
provs_df['Median Scale'] = [4, 3, 3, 3, 3, 4, 3, 4, 5, 4, 5, 5, 3] # This is lame :( Should make it calculable
provs_df 

,Province Id,Province Name,Latitude,Longitude,Total Income,Total Population,Median Per Capita among Province FEDs,Population,Per Capita,Average Per Capita Income,Avg Scale,Median FED PCI,Median Scale
0,10,Newfoundland and Labrador,51.039105,-56.715888,1.917592e+10,514536,37663.745973,"514,536",37268.381610,"$37,268",4,"$37,664",4
1,11,Prince Edward Island,46.476074,-63.312972,4.704650e+09,140204,34233.088988,"140,204",33555.747340,"$33,556",3,"$34,233",3
2,12,Nova Scotia,45.262660,-62.808875,3.128133e+10,921727,31776.197434,"921,727",33937.739699,"$33,938",3,"$31,776",3
3,13,New Brunswick,46.475575,-66.013411,2.441427e+10,751171,33058.911963,"751,171",32501.615478,"$32,502",3,"$33,059",3
4,24,Quebec,49.696473,-71.909004,2.797480e+11,7903001,34267.760452,"7,903,001",35397.692598,"$35,398",3,"$34,268",3
5,35,Ontario,47.527625,-83.360554,5.181170e+11,12851821,38060.966499,"12,851,821",40314.676029,"$40,315",4,"$38,061",4
6,46,Manitoba,52.434901,-97.963706,4.194699e+10,1208268,35317.490700,"1,208,268",34716.628265,"$34,717",3,"$35,317",3
7,47,Saskatchewan,53.068231,-106.041076,4.067702e+10,1033381,38661.654419,"1,033,381",39363.040350,"$39,363",4,"$38,662",4
8,48,Alberta,53.590719,-114.652379,1.765550e+11,3645257,45445.585526,"3,645,257",48434.170759,"$48,434",5,"$45,446",5
9,59,British Columbia,52.476869,-123.534440,1.800390e+11,4400057,39062.477225,"4,400,057",40917.424479,"$40,917",4,"$39,062",4


In [8]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width':  '900px',
    'height': '675px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
center=(43.6551165, -79.3869946) # Downtown Toronto

fig = gmaps.figure(layout=figure_layout, center=center, zoom_level=10)

In [9]:
# The FEDs layer with symbols

# FEDs central points
locations = feds_df[['Latitude', 'Longitude']]
hover_text = feds_df['Average Per Capita Income'].tolist()
fill_color = make_color_list(feds_df['Party Id'])
stroke_color = fill_color
scale = make_scale_list(feds_df.loc[:, 'Per Capita'])

data_layer = gmaps.symbol_layer(
    locations,
    hover_text=hover_text, # Unfortunately, this parameter is just ignored
    fill_color=fill_color, stroke_color=stroke_color,
    scale=scale
)

# Add the layer to the map
fig.add_layer(data_layer)

In [10]:
# The Provinces layer with markers and infoboxes

# Province center points
center_points = provs_df[['Latitude', 'Longitude']]

# Using the template to add info boxes
info_box_template = """
<dl>
<dt>Province</dt><dd>{Province Name}</dd>
<dt>Population</dt><dd>{Population}</dd>
<dt>Average Per Capita Income</dt><dd><img src="scale_{Avg Scale}.png"/>{Average Per Capita Income}</dd>
<dt>Median FED PCI</dt><dd><img src="scale_{Median Scale}.png"/>{Median FED PCI}</dd>
</dl>
"""
info_boxes = [info_box_template.format(**row) for index, row in provs_df.iterrows()]

markers = gmaps.marker_layer(center_points, info_box_content=info_boxes)

# Add the layer to the map
fig.add_layer(markers)

In [11]:
# Display
fig

Figure(layout=FigureLayout(border='1px solid black', height='675px', margin='0 auto 0 auto', padding='1px', wi…